# MidTerm Project: NSMC(Naver Sentiment Movie Corpus) 분류

- rating_train.txt는 네이버 영화평을 0 (negative), 1(positive)로 분류해 놓은 자료이고, ratings_test.txt는 같은 영화평의 테스트용 데이터이다.
- 이 파일을 가지고 https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라  nsmc 분류기를 만들어라
- training data에서 evaluation data를 나누어 사용할 수 있다.(필요시)
- training data에 나오는 영화평을 가지고 word2vec이나, FastText 등의 임베딩을 만들고 이를 테스트시 사용하라
- 제출은 주피터 노트북과 학습된 임베딩 파일 (임베딩 파일이 커서 etl에 탑재가 되지 않으면 압축을 하거나 이메일 등 다른 방법으로 제출)
- 화일 이름은 MidTermAssignment_학번_이름
- 마감: 2023년 10월 31일 화요일 오후 11시 59분 59초까지!

## 목표

- csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기 (**새로운 torchtext사용**)
- 이미 실습 시간에 관련 모듈 설명됨
- 직접 학습한 한국어 단어 임베딩을 torchtext에 통합하여 사용하기
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 기사들의 분류 결과를 보이기
- 참고 사이트:
- [Pytorch Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis)에 관련 코드들이 있어 참조할 수 있음

## 주의사항

- 방법
1. torchtext 최신 버전 0.14.0으로 작업.
- [TorchText 최근버전 documenation](https://pytorch.org/text/0.14.0/index.html)
2. 단어 임베딩은 Gensim을 사용하여 자유롭게 구축할 수 있음(차원, 윈도우크기, 학습율 등을 자유롭게 설정)


## 전체 구현 정리
- 임베딩 방법 자세히 기술
- 데이터 전처리 자세히 기술
- rnn, lstm, cnn 등의 방법으로 기술한 것의 성능 차이 기술
- 테스트 데이터의 성능 보고
- predict한 문장들의 성능 및 분석


In [ ]:
##프로그래밍 시작
## 반드시 각 모듈별로 자세히 주석을 붙이고 설명을 할 것!

# 0. 필요한 모듈 다운로드 및  import, 파일 경로(구글 드라이브 마운트)설정 

In [38]:
!pip install konlpy
!pip install mecab-python3
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
openjdk-8-jdk is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 new

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Mecab
import torch
import torchtext
from sklearn.model_selection import train_test_split
import random
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import Vectors
from torchtext.vocab import vocab
from torchtext import data
from torchtext.data.functional import to_map_style_dataset
import numpy as np
# from torchtext.legacy import datasets
# from torchtext.legacy.data import Iterator, BucketIterator


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Vocab 만들기
-cnn_with_embedding.ipynb에 자세히 설명이 기술되어 있기에 생략한다.

In [41]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
from torchtext.vocab import build_vocab_from_iterator
train_file_path = '/content/drive/MyDrive/nlp/ratings_train.txt'
test_file_path = '/content/drive/MyDrive/nlp/ratings_test.txt'

data_pipe = dp.iter.IterableWrapper([train_file_path])
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)

In [42]:
for sample in data_pipe:
    print(sample)
    break

('9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0')


In [43]:
def removeAttribution(row):
    return row[1:3]
data_pipe = data_pipe.map(removeAttribution)

In [44]:
for sample in data_pipe:
    print(sample)
    break

('아 더빙.. 진짜 짜증나네요 목소리', '0')


In [45]:
import re

mecab=Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
def tokenizer(text):
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글과 공백만 남기고 나머지 제거
    text = re.sub('^ +', '', text)  # 시작 부분의 여러 개의 연속된 공백 제거
    tokens = mecab.morphs(text)
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

In [46]:
tokenizer("아 더빙.. 진짜 짜증나네요 목소리")

['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리']

In [47]:
def yield_tokens(data_iter):
    for text, label in data_iter:
        yield tokenizer(text)

In [48]:
vocab = build_vocab_from_iterator(yield_tokens(data_pipe), min_freq=2,
                                  specials=["<unk>", "<pad>"],special_first=True,
                                  max_tokens= 25000)
vocab.set_default_index(vocab["<unk>"])

In [49]:
print(vocab.get_itos()[:100])

['<unk>', '<pad>', '영화', '다', '고', '하', '을', '보', '게', '지', '있', '없', '좋', '나', '었', '만', '는데', '너무', '봤', '적', '안', '로', '정말', '음', '것', '아', '재밌', '네요', '어', '지만', '같', '진짜', '에서', '기', '했', '네', '점', '않', '거', '았', '수', '되', '면', 'ㅋㅋ', '인', '말', '연기', '최고', '주', '내', '평점', '이런', '던', '어요', '할', '왜', '겠', '해', '스토리', 'ㅋㅋㅋ', '습니다', '듯', '아니', '드라마', '생각', '더', '그', '싶', '사람', '감동', '때', '함', '배우', '본', '까지', '보다', '뭐', '볼', '알', '만들', '내용', '감독', '라', '재미', '그냥', '시간', '재미있', '지루', '중', '잼', '재미없', '였', '년', '쓰레기', '사랑', '못', '냐', '서', '라고', '니']


In [50]:
cnt = 0
for i in data_pipe:
    print(i)
    cnt +=1
    if cnt == 10:
      break

('아 더빙.. 진짜 짜증나네요 목소리', '0')
('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1')
('너무재밓었다그래서보는것을추천한다', '0')
('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0')
('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1')
('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0')
('원작의 긴장감을 제대로 살려내지못했다.', '0')
('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0')
('액션이 없는데도 재미 있는 몇안되는 영화', '1')
('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1')


# 2. Dataloader 만들기
-cnn_with_embedding.ipynb에 자세히 설명이 기술되어 있기에 생략한다.

In [51]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [52]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


cnn과 다른 부분이 있어 추가 설명한다. cnn에서는 없었던 text_len를 추가해줬다.

In [53]:
#collate_function: process the list of samples to form a batch.
# https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/

def lstm_custom_collate_fn(batch):
    label_list, text_list,text_len = [], [], []
    for (_text,_label) in batch:
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         if(len(processed_text)==0):
            continue
         label_list.append(label_pipeline(_label))
         text_list.append(processed_text)
         text_len.append(len(processed_text))

    label_list = torch.tensor(label_list, dtype=torch.int64).unsqueeze(dim=1)
    text_list = pad_sequence(text_list, padding_value = 1)
    return text_list.to(device),label_list.to(device),text_len

In [54]:
data_list = list(data_pipe)

# 데이터를 train set과 validation set으로 나눔
train_data, valid_data = train_test_split(data_list, test_size=0.2, random_state=42)

In [55]:
data_list[0:10]

[('아 더빙.. 진짜 짜증나네요 목소리', '0'),
 ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'),
 ('너무재밓었다그래서보는것을추천한다', '0'),
 ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'),
 ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'),
 ('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'),
 ('원작의 긴장감을 제대로 살려내지못했다.', '0'),
 ('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'),
 ('액션이 없는데도 재미 있는 몇안되는 영화', '1'),
 ('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1')]

In [56]:
lstm_train_dataloader = DataLoader(train_data, batch_size=32,
                              shuffle=True, collate_fn=lstm_custom_collate_fn)

lstm_valid_dataloader = DataLoader(valid_data, batch_size=32,
                              shuffle=True, collate_fn=lstm_custom_collate_fn)


# 3. W2V embedding 모델 만들기
 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.


In [57]:
train_file_path = '/content/drive/MyDrive/nlp/ratings_train.txt'
test_file_path = '/content/drive/MyDrive/nlp/ratings_test.txt'

train = pd.read_csv(train_file_path, sep='\t')
test = pd.read_csv(test_file_path, sep='\t')

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])
print(len(train),len(test))

train=train.dropna()
test=test.dropna()
print(len(train),len(test))
print(train)
print(test)

150000 50000
149995 49997
                                                 document  label
0                                     아 더빙.. 진짜 짜증나네요 목소리      0
1                       흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2                                       너무재밓었다그래서보는것을추천한다      0
3                           교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4       사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...                                                   ...    ...
149995                                인간이 문제지.. 소는 뭔죄인가..      0
149996                                      평점이 너무 낮아서...      1
149997                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[149995 rows x 2 columns]
                                                document  label
0                                                    굳 ㅋ      1
1                                   GDN

In [58]:
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

train_data, valid_data = train_test_split(train, test_size=0.2, random_state=42)
print(len(train),len(train_data),len(valid_data))

<ipython-input-58-1e0fb836acdb>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


149995 119996 29999


In [59]:
stopwords = ['의','가','이','은','들','는','과','도','을','를','으로','자','에','와','한','하다']

KERNEL_SIZE = [3,4,5] # 총 3개의 kernel size 사용( KERNEL_SIZE, embed_dimension)
mecab=Mecab()
def tokenizer(text):
    token = mecab.morphs(text)
    if len(token) < max(KERNEL_SIZE):
        for i in range(0, max(KERNEL_SIZE)-len(token)):
            token.append('<pad>') # 커널 사이즈 보다 문장의 길이가 작은 경우 에러 방지
    return token

tokenized_train_data = train_data['document'].apply(lambda sentence: [word for word in tokenizer(sentence) if word not in stopwords])
tokenized_validation_data = valid_data['document'].apply(lambda sentence: [word for word in tokenizer(sentence) if word not in stopwords])


In [60]:
print(len(tokenized_train_data),len(train_data))
print(len(tokenized_validation_data),len(valid_data))

119996 119996
29999 29999


In [61]:
w2v_model = Word2Vec(tokenized_train_data, min_count = 2, vector_size = 100, workers = 3, window = 5, sg = 0)

In [62]:
w2v_model.save("w2v.model")

# 4. LSTM with word2vec
더 상세한 내용은 cnn_with_embedding.ipynb에 있다.여기서는 cnn_with_embedding과 다른 점만 설명한다.


In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

미리 학습된 워드 임베딩(w2v_model)을 활용한 lstm 모델을 정의했다.
trained된 embedding을 넣는 부분만 달라졌다.
구체적인 모델은 수업시간에 배운대로 정의했다.

In [64]:
import torch.nn as nn
import gensim
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout,pad_idx):

        super().__init__()
        #여기서 아까 만든 w2v 모델을 넣는다.
        w2vmodel = gensim.models.KeyedVectors.load('w2v.model')
        weights = w2vmodel.wv
        # With pretrained embeddings
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights.vectors), padding_idx=pad_idx)


        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(dropout)
    #text_len이 추가되었음에 유의하자
    def forward(self, text, text_len):

        #text = [sent len, batch size]

        embedded = self.dropout(self.embedding(text))

        #embedded = [sent len, batch size, emb dim]

        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths = text_len, enforce_sorted = False)


        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors

        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]

        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        #hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)

In [65]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX =  w2v_model.wv.key_to_index['<pad>']

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,310,657 trainable parameters


# 5. 생성한 lstm을 Training

 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 자세한 내용은 생략한다.


In [66]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

text_len이 추가되어 있ㅇ므을 유의해서 함수를 작성한다.

In [ ]:
import time


def train(dataloader):
    model.train()
    epoch_loss, epoch_acc = 0, 0
    log_interval = len(dataloader)
    start_time = time.time()

    #text_len이 추가되었음에 유의해야 한다.
    for idx, (text, label, text_len) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, text_len)
        loss = criterion(predicted_label, label.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        epoch_acc += binary_accuracy(predicted_label,label).item()
        epoch_loss += loss.item()
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                            epoch_loss/len(dataloader)))
            #total_acc, total_count = 0, 0
            start_time = time.time()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)



In [67]:
def evaluate(dataloader):
    model.eval()
    epoch_loss, epoch_acc = 0, 0
    with torch.no_grad():
        for idx, (text, label, text_len) in enumerate(dataloader):
            predicted_label = model(text, text_len)
            loss = criterion(predicted_label, label.float())
            epoch_acc += binary_accuracy(predicted_label,label).item()
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [68]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 10

best_valid_loss = float('inf')
for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    start_time = time.time()

    train_loss, train_acc = train(lstm_train_dataloader)
    valid_loss, valid_acc = evaluate(lstm_valid_dataloader)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm_with_w2v.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 12s
	Train Loss: 0.536 | Train Acc: 71.62%
	 Val. Loss: 0.446 |  Val. Acc: 78.56%
Epoch: 02 | Epoch Time: 1m 11s
	Train Loss: 0.461 | Train Acc: 77.01%
	 Val. Loss: 0.416 |  Val. Acc: 80.39%
Epoch: 03 | Epoch Time: 1m 11s
	Train Loss: 0.436 | Train Acc: 78.76%
	 Val. Loss: 0.395 |  Val. Acc: 81.37%
Epoch: 04 | Epoch Time: 1m 8s
	Train Loss: 0.419 | Train Acc: 79.83%
	 Val. Loss: 0.390 |  Val. Acc: 81.86%
Epoch: 05 | Epoch Time: 1m 9s
	Train Loss: 0.409 | Train Acc: 80.58%
	 Val. Loss: 0.380 |  Val. Acc: 82.20%
Epoch: 06 | Epoch Time: 1m 9s
	Train Loss: 0.399 | Train Acc: 81.05%
	 Val. Loss: 0.382 |  Val. Acc: 82.63%
Epoch: 07 | Epoch Time: 1m 9s
	Train Loss: 0.395 | Train Acc: 81.30%
	 Val. Loss: 0.374 |  Val. Acc: 82.79%
Epoch: 08 | Epoch Time: 1m 10s
	Train Loss: 0.389 | Train Acc: 81.59%
	 Val. Loss: 0.378 |  Val. Acc: 83.02%
Epoch: 09 | Epoch Time: 1m 9s
	Train Loss: 0.384 | Train Acc: 81.91%
	 Val. Loss: 0.369 |  Val. Acc: 83.12%
Epoch: 10 | Epoch Time: 

# 6. 테스트 데이터 evalute 
 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.


In [ ]:
test_file_path = '/content/drive/MyDrive/nlp/ratings_test.txt'

test_data_pipe = dp.iter.IterableWrapper([test_file_path])
test_data_pipe = dp.iter.FileOpener(test_data_pipe, mode='rb')
test_data_pipe = test_data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)
test_data_pipe = test_data_pipe.map(removeAttribution)

test_dataloader = DataLoader(list(test_data_pipe), batch_size=16,
                              shuffle=True, collate_fn=lstm_custom_collate_fn)
model.eval()
test_loss, test_acc = evaluate(test_dataloader)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.380 | Test Acc: 82.96%


#7. Inference
 cnn_with_embedding.ipynb에 상세한 내용이 있으니까 생략한다.

- 다음 문장들을 모델에 넣었을 때 그 결과를 도출하는 inference 작성
- 0(부정)/1(긍정)
- 아래 문장의 정답은 1/1/0/0/1/1/1/0/0/0


In [69]:
#model.load_state_dict(torch.load('/content/lstm_with_w2v.pt'))

<All keys matched successfully>

In [70]:
def predict_nsmc(model, sentence, min_len=5):
  threshold=0.8
  hey=[(sentence,'0')]
  hey_dataloader = DataLoader(hey, batch_size=1,
                                shuffle=True, collate_fn=lstm_custom_collate_fn)
  model.eval()
  for idx, (text, label, text_len) in enumerate(hey_dataloader):
        prediction = torch.sigmoid(model(text, text_len))
        print(prediction.item()," : ", sentence)
        prediction = 1 if prediction.item() >= threshold else 0

  return prediction

In [72]:
result=[]
sentence="잔잔한 감동... 오래 기억에 남아요... 우리 시대의 평범한 가족 모습이 솔직하게 담겨 있네요. 그리고 한 소녀의 성장기도 아름답게 표현했습니다."
result.append(predict_nsmc(model, sentence))
sentence="한 사람의 삶을 한 화면에서 완전히 이해할 수 없다는 것을 더욱 명확하게 깨닫게 해주는 작품입니다. 밴 애플렉과 션 윌의 연기는 볼만하고, 복 받은 캐릭터들이 감동을 전해줍니다."
result.append(predict_nsmc(model, sentence))
sentence="티비 드라마보다 못한 영화. 그 당시에는 티비 드라마가 한국 영화보다 더 흥미로웠다."
result.append(predict_nsmc(model, sentence))
sentence="이 작품은 클래식을 위한 드라마나 청춘 성장 드라마와는 다른 주제를 다루고 있는데, 그 주제가 명확하지 않아 혼란스럽습니다. 현실을 고발하려는 의도는 알겠지만, 이 작품은 다소 심각한 드라마 중 하나입니다. 부채도사는 어디로..ㅠㅠ"
result.append(predict_nsmc(model, sentence))
sentence="이 작품을 통해 그들의 쾌락적인 삶을 엿볼 수 있고, 우리는 다른 형태의 쾌락을 느낍니다. 그들이 추잡하게 보일 수 있지만, 결국 우리도 돈을 추구하며 부유한 삶을 상상하는 것이 현실입니다. 이 영화는 감독의 메시지를 느끼게 합니다."
result.append(predict_nsmc(model, sentence))
sentence="미식축구를 다시 생각하게 만든 감동적인 영화입니다. 개인적으로 감명 깊었습니다."
result.append(predict_nsmc(model, sentence))
sentence="의외로 흥미로운 영화로, 지루하지 않은 사극 멜로였습니다. 캐릭터들이 매력적으로 표현되었어요."
result.append(predict_nsmc(model, sentence))
sentence="캐릭터들 중에서 할아버지를 제외하고는 짜증을 유발하는 매력적인 캐릭터가 없다는 점이 아쉽습니다."
result.append(predict_nsmc(model, sentence))
sentence="엔딩 임팩트가 부족합니다. 더 흥미로웠다면 좋았을 것 같아요."
result.append(predict_nsmc(model, sentence))
sentence="주연배우들이 지루하고, 압축된 스토리가 조금 부족한 것 같습니다. 으윽."
result.append(predict_nsmc(model, sentence))

print(result)

0.9975007176399231  :  잔잔한 감동... 오래 기억에 남아요... 우리 시대의 평범한 가족 모습이 솔직하게 담겨 있네요. 그리고 한 소녀의 성장기도 아름답게 표현했습니다.
0.9771487712860107  :  한 사람의 삶을 한 화면에서 완전히 이해할 수 없다는 것을 더욱 명확하게 깨닫게 해주는 작품입니다. 밴 애플렉과 션 윌의 연기는 볼만하고, 복 받은 캐릭터들이 감동을 전해줍니다.
0.28178760409355164  :  티비 드라마보다 못한 영화. 그 당시에는 티비 드라마가 한국 영화보다 더 흥미로웠다.
0.9200431108474731  :  이 작품은 클래식을 위한 드라마나 청춘 성장 드라마와는 다른 주제를 다루고 있는데, 그 주제가 명확하지 않아 혼란스럽습니다. 현실을 고발하려는 의도는 알겠지만, 이 작품은 다소 심각한 드라마 중 하나입니다. 부채도사는 어디로..ㅠㅠ
0.9938376545906067  :  이 작품을 통해 그들의 쾌락적인 삶을 엿볼 수 있고, 우리는 다른 형태의 쾌락을 느낍니다. 그들이 추잡하게 보일 수 있지만, 결국 우리도 돈을 추구하며 부유한 삶을 상상하는 것이 현실입니다. 이 영화는 감독의 메시지를 느끼게 합니다.
0.9955370426177979  :  미식축구를 다시 생각하게 만든 감동적인 영화입니다. 개인적으로 감명 깊었습니다.
0.994544506072998  :  의외로 흥미로운 영화로, 지루하지 않은 사극 멜로였습니다. 캐릭터들이 매력적으로 표현되었어요.
0.5149775743484497  :  캐릭터들 중에서 할아버지를 제외하고는 짜증을 유발하는 매력적인 캐릭터가 없다는 점이 아쉽습니다.
0.7111943364143372  :  엔딩 임팩트가 부족합니다. 더 흥미로웠다면 좋았을 것 같아요.
0.011933899484574795  :  주연배우들이 지루하고, 압축된 스토리가 조금 부족한 것 같습니다. 으윽.
[1, 1, 0, 1, 1, 1, 1, 0, 0, 0]
